<a href="https://colab.research.google.com/github/LucaRo29/NLP/blob/luca/NLP_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_text 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.6 MB 14.2 MB/s 
     |████████████████████████████████| 511.7 MB 6.1 kB/s 
     |████████████████████████████████| 438 kB 65.6 MB/s 
     |████████████████████████████████| 5.8 MB 58.4 MB/s 
     |████████████████████████████████| 1.6 MB 49.9 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Unins

In [ ]:
import keras
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from keras.callbacks import EarlyStopping
import numpy as np
import seaborn as sns

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:

df = pd.read_csv('/content/drive/MyDrive/enron_prep.csv', index_col=False)

new_model = True


# check count and unique and top values and their frequency
df['label'].value_counts()

# creating 2 new dataframe as df_ham , df_spam
df_spam = df[df['label'] == 1]
df_ham = df[df['label'] == 0]

df_ham_downsampled = df_ham.sample(df_spam.shape[0])


# concating both dataset - df_spam and df_ham_balanced to create df_balanced dataset
df_balanced = pd.concat([df_spam, df_ham_downsampled])


df_balanced.dropna(inplace=True)

# df = df.sample(100)
# print('data is downsampled')

#df_balanced = df_balanced.sample(100)
X_train, X_val, y_train, y_val = train_test_split(df_balanced['transformed_text'], df_balanced['label'], train_size=0.7)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, train_size=0.5)


In [ ]:
model = None
if (new_model):

    print('Generating new model')
    bert_preprocessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
    bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='Inputs')
    preprocessed_text = bert_preprocessor(text_input)
    embed = bert_encoder(preprocessed_text)
    dropout = tf.keras.layers.Dropout(0.5, name='Dropout')(embed['pooled_output'])
    x = tf.keras.layers.Dense(64, activation='relu')(dropout)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid', name='Dense')(x)

    # creating final model
    model = tf.keras.Model(inputs=[text_input], outputs=[outputs])

    model.save("../content/Model")
else:
    print('Loading old model')
    model = keras.models.load_model("../Bert/Model")

# if (model is None):
#     print("Error: No model")
#     return

print(model.summary())

Metrics = [tf.keras.metrics.BinaryAccuracy(name='accuracy'),
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall')
            ]

# compiling our model
model.compile(optimizer='adam',loss='binary_crossentropy',  metrics=Metrics)

es = EarlyStopping(patience=5, verbose=1, min_delta=0.001, monitor='loss', mode='auto',
                       restore_best_weights=True)

num_epochs = 100
history = model.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_val, y_val),batch_size = 50,
                    callbacks=[es],shuffle= True)


if es.stopped_epoch > 0:
  num_epochs = es.stopped_epoch


# model.evaluate(X_test, y_test)


# getting y_pred by predicting over X_text and flattening it
y_pred = model.predict(X_test)
y_pred = y_pred.flatten()  # require to be in one-dimensional array , for easy manipulation

# y_test = y_test.to_numpy()
y_pred = np.where(y_pred > 0.5, 1, 0)


Generating new model


INFO:tensorflow:Assets written to: ../content/Model/assets


INFO:tensorflow:Assets written to: ../content/Model/assets


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Inputs (InputLayer)            [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['Inputs[0][0]']                 
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [ ]:
model.save('/content/drive/MyDrive/NLP/Bert_enron')


INFO:tensorflow:Assets written to: /content/drive/MyDrive/NLP/Bert_enron/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/NLP/Bert_enron/assets


In [ ]:
# creating confusion matrix

cm = confusion_matrix(y_test, y_pred)


#creating a graph out of confusion matrix
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Actual')
# plt.savefig('../Bert/Data/bert_CM.png')
plt.savefig('/content/drive/MyDrive/NLP/Bert_enron/bert_CM.png')
plt.show()

#epochs = range(1, num_epochs+1)
#print(epochs)
plt.plot(history.history['loss'], 'g', label='Training loss')
plt.plot(history.history['val_loss'], 'b', label='validation loss')
plt.axvline(x=num_epochs, color='b', label='best_epoch')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('/content/drive/MyDrive/NLP/Bert_enron/bert_loss.png')
plt.show()



#epochs = range(1, num_epochs+1)
plt.plot(history.history['precision'], 'g', label='Training precision')
plt.plot(history.history['val_precision'], 'b', label='validation precision')
plt.axvline(x=num_epochs, color='b', label='best_epoch')
plt.title('Training and Validation precision')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('/content/drive/MyDrive/NLP/Bert_enron/bert_precision.png')
# plt.savefig('../Bert/Data/bert_precision.png')
plt.show()


plt.plot( history.history['accuracy'], 'g', label='Training accuracy')
plt.plot(history.history['val_accuracy'], 'b', label='validation accuracy')
plt.axvline(x=num_epochs, color='b', label='best_epoch')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('/content/drive/MyDrive/NLP/Bert_enron/bert_accuracy.png')
# plt.savefig('../Bert/Data/bert_accuracy.png')
plt.show()

# printing classification report
print(classification_report(y_test, y_pred))

predict_text = [
    # Spam
    'We’d all like to get a $10,000 deposit on our bank accounts out of the blue, but winning a prize—especially if you’ve never entered a contest',
    'Netflix is sending you a refund of $12.99. Please reply with your bank account and routing number to verify and get your refund',
    'Your account is temporarily frozen. Please log in to to secure your account ',
    # ham
    'The article was published on 18th August itself',
    'Although we are unable to give you an exact time-frame at the moment, I would request you to stay tuned for any updates.',
    'The image you sent is a UI bug, I can check that your article is marked as regular and is not in the monetization program.'
]
test_results = model.predict(predict_text)
output = np.where(test_results > 0.5, 'spam', 'ham')
print(output)

NameError: ignored